In [1]:
import pandas as pd
import requests

# Mini Project 1 - DSAI Class Scheduling

####  Technique Used: 
* Object Oriented Programming
* One Hot Encoding,
* Genetic Algorithm,


# 1. Introduction

### 1.1. Scheduling problem

#### The problem is to schedule the timetable for DSAI classes (1, 2, 3) and IT-E15 class in a suitable timetable such that the classes can be taught:

* In appropriate rooms (which means that the room capacity should be greater than the number of students in a class and it should be of the correct type - either a laboratory or a regular classroom), 
* within suitable time slots (i.e., no two classes are scheduled in the same room during the same time slot, and to determine if the time slot is reserved for experiments), 
* With appropriate professors (meaning no professor teaches two classes at the same time). 

We will use the Genetic Algorithm to solve the problem and find a near-optimal solution.

### 1.2. Example

*Here is the example that we're going to work with:*



In [7]:
data = {'Class': ['PH1110E', 'IT1110E', 'MI2022E', 'MI1121E', 'IT1110E', 'MI2022E', 'PH1110E', 'MI1121E'],
        'Professor': ['Lê Bá Nam', 'Đinh Viết Sang', 'Nguyễn Xuân Thọ', 'Trần Ngọc Khuê', 'Lê Bá Nam', 'Nguyễn Xuân Thọ', 'Trần Ngọc Khuê', 'Đinh Viết Sang'],
        'Group': ['DSAI-01', 'DSAI-02', 'IT-E15', 'DSAI-02', 'IT-E15', 'DSAI-01', 'DSAI-01', 'DSAI-02'],
       }
# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)
df

,Class,Professor,Group
0,PH1110E,Lê Bá Nam,DSAI-01
1,IT1110E,Đinh Viết Sang,DSAI-02
2,MI2022E,Nguyễn Xuân Thọ,IT-E15
3,MI1121E,Trần Ngọc Khuê,DSAI-02
4,IT1110E,Lê Bá Nam,IT-E15
5,MI2022E,Nguyễn Xuân Thọ,DSAI-01
6,PH1110E,Trần Ngọc Khuê,DSAI-01
7,MI1121E,Đinh Viết Sang,DSAI-02


*Note: IT1110E is a subject which is needed to be studied in a laboratory.*

And here is given rooms and time slots:

In [6]:
import pandas as pd
from IPython.display import display, HTML

# Data for table 1
data_rooms = {
    'Room': ['D6-208', 'B1-303', 'D3-5-501', 'D6-107'],
    'Capacity': [100, 30, 200, 50],
    'is_lab': [False, True, False, False]
}

# Create DataFrame for table 1
df_rooms = pd.DataFrame(data_rooms)

# Data in table 2
data_time_slots = {
    'Start': ['6:45', '10:15', '10:15', '6:45', '9:20', '6:15', '8:25'],
    'End': ['10:05', '11:45', '11:45', '9:15', '11:45', '8:15', '10:05'],
    'Day': ['Tuesday', 'Tuesday', 'Wednesday', 'Thursday', 'Thursday', 'Friday', 'Friday'],
    'is_lab_slot': [False, False, False, True, False, False, False]
}

# Create DataFrame for table 2
df_time_slots = pd.DataFrame(data_time_slots)

html_code = f'''
<div style="display:flex">
  <div>
    <caption>Rooms:</caption>
    {df_rooms.to_html(index=False)}
  </div>

  <div style="margin-left: 20px">
    <caption>Time slots:</caption>
    {df_time_slots.to_html(index=False)}
  </div>
</div>
'''

display(HTML(html_code))


## 2. Solution:

#### To apply the Genetic Algorithm:  
* First, we need to model the given problem data into classes. At each class, we provide search functions (find), display strings (repr), and if necessary, a boolean variable to check whether the condition is a lab or not (is_lab, is_slot_lab). All of this is stored in Classes.py: 


In [10]:
# Group, Professor, CourseClass, Room, Slot
class Group:
    groups = None
    def __init__(self, name, size):
        self.name = name
        self.size = size

    def find(name):
        for i in range(len(Group.groups)):
            if Group.groups[i].name == name:
                return i
        return -1
    
    def __repr__(self) -> str:
        return "Group: " + self.name + ", Size: " + str(self.size)

class Professor:
    professors = None
    def __init__(self, name):
        self.name = name

    def find(name):
        for i in range(len(Professor.professors)):
            if Professor.professors[i].name == name:
                return i
        return -1
    
    def __repr__(self) -> str:
        return "Professor: " + self.name

class CourseClass:
    classes = None

    def __init__(self, code, is_lab = False):
        self.code = code
        self.is_lab = is_lab

    def find(code):
        for i in range(len(CourseClass.classes)):
            if CourseClass.classes[i].code == code:
                return i
        return -1

    def __repr__(self) -> str:
        return "CourseClass: " + self.code

class Room:
    rooms = None

    def __init__(self, name, size, is_lab = False) -> None:
        self.name = name
        self.size = size
        self.is_lab = is_lab
    def find(name):
        for i in range(len(Room.rooms)):
            if Room.rooms[i].name == name:
                return i
        return -1
    
    def __repr__(self) -> str:
        return "Room: " + self.name + ", Size: " + str(self.size)

class Slot:
    slots = None

    def __init__(self, start, end, day, is_lab_slot = False) -> None:
        self.start = start
        self.end = end
        self.day = day
        self.is_lab_slot = is_lab_slot

    def __repr__(self) -> str:
        return "Slot: " + self.start + " - " + self.end + ", Day: " + self.day

##### In the main file:

Import and initialize the needed libraries and variables:

In [ ]:
import random, copy, math
from Classes import *
from math import ceil, log2

CourseClass.classes = [CourseClass("PH1110E"), CourseClass("IT1110E", True),
                        CourseClass("MI2022E"), CourseClass("MI1121E")]
Group.groups = [Group("DSAI-01", 50), Group("IT-E15", 40), Group("DSAI-02", 70)]
Professor.professors = [Professor("Đinh Viết Sang"), Professor("Lê Bá Nam"), 
                        Professor("Nguyễn Xuân Thọ"), Professor("Trần Ngọc Khuê")]
Room.rooms = [Room("D6-208", 100), Room("B1-303", 30, True), Room("D3-5-501", 200), Room("D6-107", 50)]
Slot.slots = [Slot("6:45", "10:05", "Tuesday"), Slot("10:15", "11:45", "Tuesday"),
              
              Slot("10:15", "11:45", "Wednesday"), Slot("6:45", "9:15", "Thursday", True),
              Slot("9:20", "11:45", "Thursday"), Slot("6:15", "8:15", "Friday"), 
              Slot("8:25", "10:05", "Friday")]

# Initialize needed properties
cpg = []
rooms = []
slots = []
max_score = -1

* Then, encode the given data into binary strings. This is highly beneficial for generating variations and using them for creating mutations and creating new generations. There is a pretty cool trick to optimize the time doing this by using memoization. 

In [9]:
bits_backup = {}

def bits_needed(x):
#    global bits_backup
    r = bits_backup.get(id(x))
    if r == None:
        r = int(ceil(log2(len(x))))
        bits_backup[id(x)] = r
    return max(1, r)

def join_cpg_pair(_cpg):
    res = []
    for i in range(0, len(_cpg), 3):
        res.append(_cpg[i] + _cpg[i+1] + _cpg[i+2])
    return res

def convert_input_into_bin():
    global cpg, rooms, slots, max_score
    cpg = [CourseClass.find("PH1110E"), Professor.find("Lê Bá Nam"), Group.find("DSAI-01"),
           CourseClass.find("IT1110E"), Professor.find("Đinh Viết Sang"), Group.find("DSAI-02"),
           CourseClass.find("MI2022E"), Professor.find("Nguyễn Xuân Thọ"), Group.find("IT-E15"),
           CourseClass.find("MI1121E"), Professor.find("Trần Ngọc Khuê"), Group.find("DSAI-02"),
           CourseClass.find("IT1110E"), Professor.find("Lê Bá Nam"), Group.find("IT-E15"),
           CourseClass.find("MI2022E"), Professor.find("Nguyễn Xuân Thọ"), Group.find("DSAI-01"),
           CourseClass.find("PH1110E"), Professor.find("Trần Ngọc Khuê"), Group.find("DSAI-01"),
           CourseClass.find("MI1121E"), Professor.find("Đinh Viết Sang"), Group.find("DSAI-02"),]
# Convert each elements into a binary string
    for _c in range(len(cpg)):
        if _c % 3:  # CourseClass
            cpg[_c] = (bin(cpg[_c])[2:]).rjust(bits_needed(CourseClass.classes), '0')
        elif _c % 3 == 1:  # Professor
            cpg[_c] = (bin(cpg[_c])[2:]).rjust(bits_needed(Professor.professors), '0')
        else:  # Group
            cpg[_c] = (bin(cpg[_c])[2:]).rjust(bits_needed(Group.groups), '0')

    cpg = join_cpg_pair(cpg)
    for r in range(len(Room.rooms)):
        rooms.append((bin(r)[2:]).rjust(bits_needed(Room.rooms), '0'))

    for t in range(len(Slot.slots)):
        slots.append((bin(t)[2:]).rjust(bits_needed(Slot.slots), '0'))

    # print(cpg)
    max_score = (len(cpg) - 1) * 3 + len(cpg) * 3

* Next, we need to define the conditions for creating an ideal timetable (which is already listed in section 1.1), in other words, constructing a fitness function,

In [12]:
def course_bits(chromosomme):
    i = 0
    return chromosomme[i : i + bits_needed(CourseClass.classes)]
def professor_bits(chromosomme):
    i = bits_needed(CourseClass.classes)
    return chromosomme[i : i + bits_needed(Professor.professors)]
def group_bits(chromosomme):
    i = bits_needed(CourseClass.classes)+bits_needed(Professor.professors)
    return chromosomme[i : i + bits_needed(Group.groups)]
def slot_bits(chromosomme):
    i = bits_needed(CourseClass.classes)+bits_needed(Professor.professors)+\
        bits_needed(Group.groups)
    return chromosomme[i : i + bits_needed(Slot.slots)]
def room_bits(chromosomme):
    i = bits_needed(CourseClass.classes) + bits_needed(Professor.professors)+\
        bits_needed(Group.groups) + bits_needed(Slot.slots)
    return chromosomme[i : i + bits_needed(Room.rooms)]
# This function is just for convenience in later calculation. 
def slot_crash(i, j):
    if slot_bits(i) == slot_bits(j):
        return 1
    return 0
# Build the fitness function
def faculty_member_one_class(chromosome):
    score = 0
    
    for i in range(len(chromosome)-1):
        crash = False
        for j in range(i+1, len(chromosome)):
            if slot_crash(chromosome[i], chromosome[j])\
                   and professor_bits(chromosome[i]) == professor_bits(chromosome[j]):
                crash = True
                break
        if not crash:
            score += 1
    return score
def group_member_one_class(chromosome):
    score = 0
    
    for i in range(len(chromosome)-1):
        crash = False
        for j in range(i+1, len(chromosome)):
            if slot_crash(chromosome[i], chromosome[j])\
                and group_bits(chromosome[i]) == group_bits(chromosome[j]):
                crash = True
                break
        score += 1
    return score
# Use room available
def use_spare_room(chromosome):
    score = 0
    for i in range(len(chromosome)-1):
        crash = False
        for j in range(i+1, len(chromosome)):
            if slot_crash(chromosome[i], chromosome[j]) and room_bits(chromosome[i]) == room_bits(chromosome[j]):
                crash = True
            break
        if not crash:
            score += 1
    return score
# check if the room is large enough
def classroom_size(chromosome):
    score = 0
    for _c in chromosome:
        if Group.groups[int(group_bits(_c),2)].size <= Room.rooms[int(room_bits(_c),2)].size:
            score += 1
    return score
# check if the room is appropriate for particular class/lab
def appropriate_room(chromosome):
    score = 0
    for _c in chromosome:
        if CourseClass.classes[int(course_bits(_c),2)].is_lab == Room.rooms[int(room_bits(_c),2)].is_lab:
            score += 1
    return score
# check if that lab is allocated appropriate time slot
def appropriate_timeslot(chromosome):
    score = 0
    for _c in chromosome:
        if CourseClass.classes[int(course_bits(_c),2)].is_lab == Slot.slots[int(slot_bits(_c),2)].is_lab_slot:
            score += 1
    return score

* Finally, we use genetic algorithm as usual.

In [2]:
def evaluate(chromosomes):
    global max_score
    score = 0
    score = score + use_spare_room(chromosomes)
    score = score + faculty_member_one_class(chromosomes)
    score = score + classroom_size(chromosomes)
    score = score + group_member_one_class(chromosomes)
    score = score + appropriate_room(chromosomes)
    score = score + appropriate_timeslot(chromosomes)
    return score / max_score

def cost(solution):
    return 1 / float(evaluate(solution))
# n ở đây là số lượng cá thể ban đầu, mỗi lần lặp tạo ra một giải pháp tiềm năng
def init_population(n):
# ?    global cpg, rooms, slots
    chromosomes = []
    for i in range(n):
        chromosome = []
        for _c in cpg:
            chromosome.append(_c + random.choice(slots) + random.choice(rooms))
        chromosomes.append(chromosome)
    return chromosomes
# thay đổi một lịch trình bất kì trong giải pháp này
def mutate(chromosome):
    a = random.randint(0, len(chromosome)-1)
    b = random.choice(slots)
    c = random.choice(rooms)
# why not cpg?
    chromosome[a] = course_bits(chromosome[a]) + professor_bits(chromosome[a])+\
                    group_bits(chromosome[a]) + b + c 
# crossover and append new itinerary
def crossover(population):
    a = random.randint(0, len(population)-1)
    b = random.randint(0, len(population)-1)
# Here we assume that all solutions have the same length
    cut = random.randint(0, len(population[0])-1)
    population.append(population[a][:cut]+population[b][cut:])
# Choose only n optimal solutions for the next round 
def selection(population, n):
    population.sort(key = evaluate, reverse = True)
    while len(population) > n:
        population.pop()
# func to print solutions
def print_chromosome(chromosome):
    print(CourseClass.classes[int(course_bits(chromosome), 2)], " | ",
          Professor.professors[int(professor_bits(chromosome), 2)], " | ",
          Group.groups[int(group_bits(chromosome), 2)], " | ",
          Slot.slots[int(slot_bits(chromosome), 2)], " | ",
          Room.rooms[int(room_bits(chromosome), 2)])
def ssn(solution):
    rand_slot = random.choice(slots)
    rand_lt = random.choice(rooms)
    
    a = random.randint(0, len(solution) - 1)
    
    new_solution = copy.deepcopy(solution)
    new_solution[a] = course_bits(solution[a]) + professor_bits(solution[a]) +\
        group_bits(solution[a]) + rand_slot + room_bits(solution[a])
    return [new_solution]

# Swapping Neighborhoods
# It randomy selects two classes and swap their time slots
def swn(solution):
    a = random.randint(0, len(solution) - 1)
    b = random.randint(0, len(solution) - 1)
    new_solution = copy.deepcopy(solution)
    temp = slot_bits(solution[a])
    new_solution[a] = course_bits(solution[a]) + professor_bits(solution[a]) +\
        group_bits(solution[a]) + slot_bits(solution[b]) + room_bits(solution[a])

    new_solution[b] = course_bits(solution[b]) + professor_bits(solution[b]) +\
        group_bits(solution[b]) + temp + room_bits(solution[b])
    # print("Diff", solution)
    # print("Meiw", new_solution)
    return [new_solution]

def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost < old_cost:
        return 1.0
    else:
        return math.exp((old_cost - new_cost) / temperature)

def simulated_annealing():
    alpha = 0.9
    T = 1.0
    T_min = 0.00001
    
    convert_input_into_bin()
    population = init_population(1) # as simulated annealing is a single-state method
    old_cost = cost(population[0])
    # print("Cost of original random solution: ", old_cost)
    # print("Original population:")
    # print(population)

    for __n in range(500):
        new_solution = swn(population[0])
        new_solution = ssn(population[0])
        new_cost = cost(new_solution[0])
        ap = acceptance_probability(old_cost, new_cost, T)
        if ap > random.random():
            population = new_solution
            old_cost = new_cost
        T = T * alpha
    # print(population)
    # print("Cost of altered solution: ", cost(population[0]))
    print("\n------------- Simulated Annealing --------------\n")
    for lec in population[0]:
        print_chromosome(lec)
    print("Score: ", evaluate(population[0]))    

def genetic_algorithm():
    generation = 0

    convert_input_into_bin()
    population = init_population(3)
    while True:
            
            # if termination criteria are satisfied, stop.
            if evaluate(max(population, key=evaluate)) == 1 or generation == 500:
                print("Generations:", generation)
                print("Best Chromosome fitness value", evaluate(max(population, key=evaluate)))
                print("Best Chromosome: ", max(population, key=evaluate))
                for lec in max(population, key=evaluate):
                    print_chromosome(lec)
                break
            
            # Otherwise continue
            else:
                for _c in range(len(population)):
                    crossover(population)
                    selection(population, 5)
                    
                    # selection(population[_c], len(cpg))
                    mutate(population[_c])

            generation = generation + 1

def main():
    random.seed()
    genetic_algorithm()
    simulated_annealing()

main()

NameError: name 'random' is not defined

## 3. Conclusion:

We have the desired result, which is showed in result.img file.